## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
airlines_df = pd.read_csv("csvs/airlines.csv")
flights_df = pd.read_csv("csvs/flights.csv")
airports_df = pd.read_csv("csvs/airports.csv")


2023-03-02 20:07:30.410463: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/var/folders/vc/z_x4c0yx2cs_6rwx2wcj_sqm0000gn/T/ipykernel_55828/2020358918.py:10: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  flights_df = pd.read_csv("csvs/flights.csv")


In [2]:
print(flights_df.describe(include=['object']))

        AIRLINE TAIL_NUMBER ORIGIN_AIRPORT DESTINATION_AIRPORT  \
count   5819079     5804358        5819079             5819079   
unique       14        4897            930                 930   
top          WN      N480HA            ATL                 ATL   
freq    1261855        3768         346836              346904   

       CANCELLATION_REASON  
count                89884  
unique                   4  
top                      B  
freq                 48851  


In [3]:
# Drop the non-beneficial columns
flights = flights_df.drop(['TAIL_NUMBER', 'TAXI_OUT', 'YEAR', 'WHEELS_OFF', 'WHEELS_ON'], axis=1)
flights.head

<bound method NDFrame.head of          MONTH  DAY  DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER ORIGIN_AIRPORT  \
0            1    1            4      AS             98            ANC   
1            1    1            4      AA           2336            LAX   
2            1    1            4      US            840            SFO   
3            1    1            4      AA            258            LAX   
4            1    1            4      AS            135            SEA   
...        ...  ...          ...     ...            ...            ...   
5819074     12   31            4      B6            688            LAX   
5819075     12   31            4      B6            745            JFK   
5819076     12   31            4      B6           1503            JFK   
5819077     12   31            4      B6            333            MCO   
5819078     12   31            4      B6            839            JFK   

        DESTINATION_AIRPORT  SCHEDULED_DEPARTURE  DEPARTURE_TIME  \
0            

In [4]:
# Determine the number of unique values in each column.
flights.nunique()

MONTH                    12
DAY                      31
DAY_OF_WEEK               7
AIRLINE                  14
FLIGHT_NUMBER          6952
ORIGIN_AIRPORT          930
DESTINATION_AIRPORT     930
SCHEDULED_DEPARTURE    1321
DEPARTURE_TIME         1440
DEPARTURE_DELAY        1217
SCHEDULED_TIME          550
ELAPSED_TIME            712
AIR_TIME                675
DISTANCE               1363
TAXI_IN                 185
SCHEDULED_ARRIVAL      1435
ARRIVAL_TIME           1440
ARRIVAL_DELAY          1240
DIVERTED                  2
CANCELLED                 2
CANCELLATION_REASON       4
AIR_SYSTEM_DELAY        570
SECURITY_DELAY          154
AIRLINE_DELAY          1067
LATE_AIRCRAFT_DELAY     695
WEATHER_DELAY           632
dtype: int64

In [5]:
# Look at value counts for binning
counts = flights['AIRLINE'].value_counts()
counts

WN    1261855
DL     875881
AA     725984
OO     588353
EV     571977
UA     515723
MQ     294632
B6     267048
US     198715
AS     172521
NK     117379
F9      90836
HA      76272
VX      61903
Name: AIRLINE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of types to be replaced
# use the variable name `application_types_to_replace
flight_types_to_replace = list(counts[counts<571977].index)
flight_types_to_replace

['UA', 'MQ', 'B6', 'US', 'AS', 'NK', 'F9', 'HA', 'VX']

In [7]:
# Replace in dataframe
for airline in flight_types_to_replace:
    flights['AIRLINE'] = flights['AIRLINE'].replace(airline,"Other")

# Check to make sure binning was successful
flights['AIRLINE'].value_counts()

Other    1795029
WN       1261855
DL        875881
AA        725984
OO        588353
EV        571977
Name: AIRLINE, dtype: int64

In [8]:
# Look at DAY value counts for binning
day_counts = flights['DAY'].value_counts()
day_counts

2     195986
16    195899
20    195707
13    195089
9     194224
8     193964
23    193560
19    193284
15    192950
22    192725
27    191920
28    191401
18    191393
17    191319
6     191232
4     190893
12    190872
11    190756
3     190007
5     189766
1     189477
21    189413
10    189288
14    188611
7     187598
26    187387
25    187317
24    185017
29    179441
30    178771
31    103812
Name: DAY, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts 
flights_to_replace = list(counts[counts<195089].index)
flights_to_replace

['AS', 'NK', 'F9', 'HA', 'VX']

In [ ]:
# Convert categorical data to numeric 
flights = pd.get_dummies(flights,dtype=float)
flights.head()

In [ ]:
y = application_df['IS_SUCCESSFUL'].values
y

In [ ]:
X = application_df.drop('IS_SUCCESSFUL', axis=1).values
X

In [ ]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len( X_train_scaled[0])
hidden_nodes_layer1=7
hidden_nodes_layer2=14
hidden_nodes_layer3=21
nn = tf.keras.models.Sequential()

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.15, epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")